In [1]:


import pandas as pd
from sklearn.model_selection import KFold
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

datasets="/home/ymachta/Challenges_and_fun/Qube/Datasets/"
df = pd.read_csv(datasets+'X_imputed_DE.csv').drop(['ID',"DAY_ID","DE_RAIN","DE_WIND","DE_TEMP"], axis=1)
labels=pd.read_csv(datasets+'Y_imputed_DE.csv').drop('ID', axis=1)
labels["Rank_2"]=labels["Rank"]**2

# df_FR = pd.read_csv(datasets+'X_imputed_FR.csv').drop(['ID',"DAY_ID"], axis=1)
df_FR = pd.read_csv(datasets+'X_imputed_FR.csv').drop(['ID',"DAY_ID","FR_DE_EXCHANGE","FR_NET_EXPORT","FR_RAIN","FR_WIND","FR_TEMP"], axis=1)
labels_FR=pd.read_csv(datasets+'Y_imputed_FR.csv').drop('ID', axis=1)

list(df.columns)

['DE_CONSUMPTION',
 'DE_FR_EXCHANGE',
 'DE_NET_EXPORT',
 'DE_GAS',
 'DE_COAL',
 'DE_HYDRO',
 'DE_NUCLEAR',
 'DE_SOLAR',
 'DE_WINDPOW',
 'DE_LIGNITE',
 'DE_RESIDUAL_LOAD',
 'GAS_RET',
 'COAL_RET',
 'CARBON_RET']

In [3]:
from utils.Brute_force_feature_engineering import *

base_columns=list(df.columns)
df_copy,corr=main_brute(df,base_columns,labels,n_uplet=20,thresh=0.25)
print(len(corr))
df_copy2=df_copy[list(corr.index)+base_columns]

powers added
16369 colunms to add


  0%|          | 0/16369 [00:00<?, ?it/s]

100%|██████████| 16369/16369 [00:53<00:00, 307.69it/s]


n_uplet added
54


In [198]:
base_columns_FR=list(df_FR.columns)
df_copy_FR,corr_FR=main_brute(df_FR,base_columns_FR,labels_FR,n_uplet=9,thresh=0.13)
print(corr_FR)
df_copy2_FR=df_copy_FR[list(corr_FR.index)+base_columns_FR]

powers added
2024 colunms to add


100%|██████████| 2024/2024 [00:03<00:00, 529.12it/s]


n_uplet added
_x_GAS_RET_x_FR_WINDPOW                                                                           0.153600
_x_CARBON_RET_x_GAS_RET                                                                           0.150299
_x_CARBON_RET_x_GAS_RET_x_FR_RESIDUAL_LOAD_x_FR_GAS                                               0.142039
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_CONSUMPTION                                              -0.132993
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_NUCLEAR_x_FR_HYDRO_x_FR_GAS                              -0.133298
_x_CARBON_RET_x_GAS_RET_x_FR_RESIDUAL_LOAD_x_FR_SOLAR_x_FR_NUCLEAR_x_FR_HYDRO_x_FR_CONSUMPTION   -0.133704
_x_CARBON_RET_x_GAS_RET_x_FR_RESIDUAL_LOAD_x_FR_SOLAR_x_FR_HYDRO_x_FR_GAS_x_FR_CONSUMPTION       -0.133785
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_HYDRO_x_FR_GAS                                           -0.135605
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_NUCLEAR_x_FR_GAS_x_FR_CONSUMPTION                        -0.136024
_x_CARBON_RET_x_GAS_RET

In [200]:
inf_mask = df_copy.isin([np.inf])
df_copy.loc[inf_mask.any(axis=1), inf_mask.any(axis=0)]

""


In [201]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train_clean_DE = df
Y_train_clean_DE = labels
X_train_clean_FR = df_FR
Y_train_clean_FR = labels_FR
X_train_clean_FR, X_test_FR, Y_train_clean_FR, y_test_FR = train_test_split(X_train_clean_FR, Y_train_clean_FR, test_size=0.25)
X_train_clean_DE, X_test_DE, Y_train_clean_DE, y_test_DE = train_test_split(X_train_clean_DE, Y_train_clean_DE, test_size=0.25)

# print (output_train[:,1])
def metric_train(output,df):
    term=output[0]
    if not isinstance(term, np.ndarray):
        return  100 *spearmanr(output, df["TARGET"]).correlation
    else:
        return  [100 *spearmanr(output[:,i], df["TARGET"]).correlation for i in range(len(term)) ]

In [202]:
def average_stats(df,labels,regressor,columns=['TARGET','Rank'],iterations=20,depth=None,group=False):
    valid_list=[]
    for i in range(iterations):
        X_train_clean_DE, X_test_DE, Y_train_clean_DE, y_test_DE = train_test_split(df, labels, test_size=0.25)
        if group: 
            Y_train_clean_DE['Rank_group']=pd.qcut(Y_train_clean_DE.Rank,30).cat.codes
            if columns is None:
                columns= 'Rank_group'
            elif 'Rank_group'  not in columns: 
                if isinstance(columns,list):
                    columns=columns+['Rank_group']
                else:
                    columns=[columns]+['Rank_group']
        if depth is None:
            DF = regressor()
        else:            
            DF = regressor(max_depth=depth)
        #   Fit the ridge regressor

        DF.fit(X_train_clean_DE, Y_train_clean_DE[columns])
        output_train = DF.predict(X_test_DE)
        # print(output_train)
        validation=metric_train(output_train,y_test_DE)
        
        valid_list+=[validation]
    
    if isinstance(columns,list):
        for i,column in enumerate(columns):
            print( f"average {column} score for ", sum([row[i] for row in valid_list])/len(valid_list) )
    else:
        print( f"average {columns} score for ", sum(valid_list)/len(valid_list) )
    
    print("\n")


In [204]:

from sklearn.ensemble import RandomForestRegressor

average_stats(df_copy2,labels,RandomForestRegressor,depth=3)
average_stats(df_copy2,labels,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df_copy2,labels,RandomForestRegressor,columns='TARGET',depth=3)

average TARGET score for  32.352674136333746
average Rank score for  32.84419846055667


average Rank score for  34.93084012878229


average TARGET score for  26.659680594798015




In [205]:
average_stats(df,labels,RandomForestRegressor,depth=3)
average_stats(df,labels,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df,labels,RandomForestRegressor,columns='TARGET',depth=3)


average TARGET score for  32.75692439453148
average Rank score for  33.262689952599224


average Rank score for  34.76429066110044


average TARGET score for  25.265252729714547




In [206]:
average_stats(df,labels,RandomForestRegressor,columns=None,depth=3,group=True)
average_stats(df_copy2,labels,RandomForestRegressor,columns=None,depth=3,group=True)

average Rank_group score for  36.30638250694482


average Rank_group score for  30.68978835793998




In [207]:
average_stats(df_FR,labels_FR,RandomForestRegressor,depth=3)
average_stats(df_FR,labels_FR,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df_FR,labels_FR,RandomForestRegressor,columns='TARGET',depth=3)

average TARGET score for  17.524418953781517
average Rank score for  19.002513666718052


average Rank score for  18.585382918594313


average TARGET score for  8.441262153374547




In [208]:
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,depth=3)
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,columns='TARGET',depth=3)

average TARGET score for  19.429634761550854
average Rank score for  20.560434314123267


average Rank score for  20.69651055626914


average TARGET score for  14.259310752491492




In [209]:
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,columns=None,depth=3,group=True)
average_stats(df_FR,labels_FR,RandomForestRegressor,columns=None,depth=3,group=True)

average Rank_group score for  18.46192922678766


average Rank_group score for  17.67332174097308




In [210]:
from mord import OrdinalRidge,LAD
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    average_stats(df_copy2,labels,OrdinalRidge,iterations=100)
    average_stats(df_copy2,labels,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df_copy2,labels,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df_copy2,labels,OrdinalRidge,columns=None, group=True,iterations=100)
    
    
    average_stats(df,labels,OrdinalRidge,iterations=100)
    average_stats(df,labels,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df,labels,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df,labels,OrdinalRidge,columns=None, group=True,iterations=100)
    

average TARGET score for  10.874659238669448
average Rank score for  30.935364068064494


average Rank score for  29.5422527335191


average TARGET score for  11.008569212319385


average Rank_group score for  29.886952490934064


average TARGET score for  19.432688024490634
average Rank score for  37.99639649742855


average Rank score for  37.43823405406133


average TARGET score for  18.25836124093441


average Rank_group score for  37.61988403737143




In [211]:
from mord import OrdinalRidge,LAD
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,iterations=100)
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,columns=None, group=True,iterations=100)
    
    
    average_stats(df_FR,labels_FR,OrdinalRidge,iterations=100)
    average_stats(df_FR,labels_FR,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df_FR,labels_FR,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df_FR,labels_FR,OrdinalRidge,columns=None, group=True,iterations=100)
    

average TARGET score for  nan
average Rank score for  16.681366495657585


average Rank score for  15.245328766098751


average TARGET score for  3.483115463774554


average Rank_group score for  16.927773299304082


average TARGET score for  nan
average Rank score for  18.802887219049854


average Rank score for  19.980412204254016


average TARGET score for  nan


average Rank_group score for  17.812501983223473




In [175]:
Y_train_clean_DE['Rank_group']=pd.qcut(Y_train_clean_DE.Rank,31).cat.codes

Y_train_clean_FR['Rank_group']=pd.qcut(Y_train_clean_FR.Rank,10).cat.codes

In [176]:
Y_train_clean_DE['Rank_group']

583     3
267     9
333    19
92      3
173    30
       ..
324    26
221    18
132     1
165     6
355    28
Name: Rank_group, Length: 447, dtype: int8

In [16]:
from typing import Dict, Tuple

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_ranking as tfr
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [18]:
movies = movies.map(lambda x: x["movie_title"])
users = ratings.map(lambda x: x["user_id"])

In [19]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(
    mask_token=None)
user_ids_vocabulary.adapt(users.batch(1000))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(
    mask_token=None)
movie_titles_vocabulary.adapt(movies.batch(1000))

In [32]:
key_func = lambda x: user_ids_vocabulary(x["user_id"])
reduce_func = lambda key, dataset: dataset.batch(100)
ds_train = ratings.group_by_window(
    key_func=key_func, reduce_func=reduce_func, window_size=100)

In [33]:
for x in ds_train.take(1):
  for key, value in x.items():
    print(f"Shape of {key}: {value.shape}")
    print(f"Example values of {key}: {value[:5].numpy()}")
    print()

Shape of movie_title: (100,)
Example values of movie_title: [b'Man Who Would Be King, The (1975)' b'Silence of the Lambs, The (1991)'
 b'Next Karate Kid, The (1994)' b'2001: A Space Odyssey (1968)'
 b'Usual Suspects, The (1995)']

Shape of user_id: (100,)
Example values of user_id: [b'405' b'405' b'405' b'405' b'405']

Shape of user_rating: (100,)
Example values of user_rating: [1. 4. 1. 5. 5.]



In [34]:
def _features_and_labels(
    x: Dict[str, tf.Tensor]) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
  labels = x.pop("user_rating")
  return x, labels


ds_train2 = ds_train.map(_features_and_labels)

ds_train2 = ds_train2.apply(
    tf.data.experimental.dense_to_ragged_batch(batch_size=32))

In [35]:
for x, label in ds_train2.take(1):
  for key, value in x.items():
    print(f"Shape of {key}: {value.shape}")
    print(f"Example values of {key}: {value[:3, :3].numpy()}")
    print()
  print(f"Shape of label: {label.shape}")
  print(f"Example values of label: {label[:3, :3].numpy()}")

Shape of movie_title: (32, None)
Example values of movie_title: [[b'Man Who Would Be King, The (1975)'
  b'Silence of the Lambs, The (1991)' b'Next Karate Kid, The (1994)']
 [b'Flower of My Secret, The (Flor de mi secreto, La) (1995)'
  b'Little Princess, The (1939)' b'Time to Kill, A (1996)']
 [b'Kundun (1997)' b'Scream (1996)' b'Power 98 (1995)']]

Shape of user_id: (32, None)
Example values of user_id: [[b'405' b'405' b'405']
 [b'655' b'655' b'655']
 [b'13' b'13' b'13']]

Shape of label: (32, None)
Example values of label: [[1. 4. 1.]
 [3. 3. 3.]
 [5. 1. 1.]]


In [39]:
class MovieLensRankingModel(tf.keras.Model):

  def __init__(self, user_vocab, movie_vocab):
    super().__init__()

    # Set up user and movie vocabulary and embedding.
    self.user_vocab = user_vocab
    self.movie_vocab = movie_vocab
    self.user_embed = tf.keras.layers.Embedding(user_vocab.vocabulary_size(),
                                                64)
    self.movie_embed = tf.keras.layers.Embedding(movie_vocab.vocabulary_size(),
                                                 64)

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    # Define how the ranking scores are computed: 
    # Take the dot-product of the user embeddings with the movie embeddings.

    user_embeddings = self.user_embed(self.user_vocab(features["user_id"]))
    movie_embeddings = self.movie_embed(
        self.movie_vocab(features["movie_title"]))
    print(user_embeddings * movie_embeddings)
    return tf.reduce_sum(user_embeddings * movie_embeddings, axis=2)

In [37]:
# Create the ranking model, trained with a ranking loss and evaluated with
# ranking metrics.
model = MovieLensRankingModel(user_ids_vocabulary, movie_titles_vocabulary)
optimizer = tf.keras.optimizers.Adagrad(0.5)
loss = tfr.keras.losses.get(
    loss=tfr.keras.losses.RankingLossKey.SOFTMAX_LOSS, ragged=True)
eval_metrics = [
    tfr.keras.metrics.get(key="ndcg", name="metric/ndcg", ragged=True),
    tfr.keras.metrics.get(key="mrr", name="metric/mrr", ragged=True)
]
model.compile(optimizer=optimizer, loss=loss, metrics=eval_metrics)

In [40]:
model.fit(ds_train, epochs=3)

Epoch 1/3


TypeError: 'NoneType' object is not callable

ValueError: Invalid reduction dimension 2 for input with 2 dimensions. for '{{node movie_lens_ranking_model/Sum}} = Sum[T=DT_FLOAT, Tidx=DT_INT32, keep_dims=false](movie_lens_ranking_model/mul, movie_lens_ranking_model/Sum/reduction_indices)' with input shapes: [?,64], [] and with computed input tensors: input[1] = <2>.\